In [ ]:
from zhipuai import ZhipuAI
import json
client = ZhipuAI(api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc") # 请填写您自己的APIKey
import mysql.connector

def execute_sql_query(sql_query:str):
    connection = mysql.connector.connect(
        host="gz-cdb-5scrcjb5.sql.tencentcdb.com",  # 数据库主机地址
        user="db",  # 数据库用户名
        password="dbdb905905",  # 数据库密码
        database="sele",  # 数据库名
        port=63432  # Corrected syntax for specifying the port argument
    )
    cursor = connection.cursor()
    # 执行SQL查询
    cursor.execute(sql_query)
    result = cursor.fetchall()  # 获取所有查询结果
    # 关闭连接
    cursor.close()
    connection.close()
    # 格式化结果为字符串，以便返回
    formatted_result = ", ".join([str(row) for row in result])
    return formatted_result

tools = [
        {
        "type": "function",
        "function": {
            "name": "execute_sql_query",
            "description": "执行 MySQL SQL 查询并返回结果。此功能利用预定义的表结构（table schemas），结合用户的数据输入，更好地理解根据用户输入需要查询哪些表，并根据这些表结构生成恰当的 SQL 查询语句，以满足用户的需求。这样可以确保生成的查询语句既符合数据库结构也符合用户查询的意图",
            "parameters": {
                "type": "object",
                "properties": {
                    "sql_query": {
                        "description": "将要在 MySQL 数据库上执行的命令语句 SQL query。此语句必须符合 SQL 语法规范，确保可以正确执行以获取预期的数据库信息。",
                        "type": "string"
                    }
                },
                "required": ["sql_query"]
            },
            "returns": {
                "description": "The result of the SQL query execution, formatted as a string.",
                "type": "string"
            }
        }
    }
]

system_message = {
    "role": "system",
    "content": "作为一个AI助手，当用户询问关于招标公告或监理招标的信息时，你会根据问题的具体内容选择恰当的数据表进行SQL查询。用户可能会使用不完整的名称或近似的文字，例如，'标'可能代表'招标公告'。对于关于招标公告的概览问题，优先在'tender_index'表中查询；若查询单个招标公告的重点信息或主要信息，则在tender_key_detail中查找；若查询单个招标公告的详细信息，则在'tender_detail'表中查找，但是尽量不要用精确文字查找，因为可能用户习惯使用部分名称而不是全称。在数据库中，日期的格式是 'YYYY-MM-DD'，例如 '2024-04-24'。查询结果将以对话形式返回，确保用户能够直观地理解查询内容和结果。"
}

def get_ai_response(user_input):
    table_schemas = {
        "tender_index": "id int, title varchar(255), publish_time date, province varchar(100), source_platform varchar(100), info_type varchar(100), business_type varchar(100), industry varchar(100), detail_link varchar(255), detail_info_fetched tinyint",
        "tender_detail": "id int, tender_id int, tender_document_start_time varchar(255), tender_document_end_time varchar(255), question_deadline varchar(255), answer_announcement_time varchar(255), bid_submission_deadline varchar(255), bid_opening_time varchar(255), tenderer varchar(255), tender_contact varchar(255), contact_phone varchar(255), tender_agency varchar(255), tender_agency_contact varchar(255), tender_agency_contact_phone varchar(255), supervision_qualification_requirement text(65535), business_license_requirement varchar(255), chief_supervisor_qualification_requirement text(65535), consortium_bidding_requirement text(65535), project_name varchar(255), investment_project_code varchar(255), tender_project_name varchar(255), implementation_site varchar(255), funding_source varchar(255), tender_scope_and_scale text(65535), duration varchar(255), maximum_bid_price varchar(255), qualification_review_method varchar(255)",
        "tender_key_detail":"tender_id int, bid_price float, construction_duration int, construction_area float, construction_cost float, qualification_type varchar(50), qualification_level varchar(50), qualification_profession varchar(100), title_level varchar(50), education varchar(50), performance_requirements varchar(100), simultaneous_projects_limit varchar(100), qualification_profession_addition varchar(100)",
        "users": "id int, openid varchar(255), session_key varchar(255), nickname varchar(255), permission int, company_id int",
    }
    messages = [
        system_message,
        {
            "role": "user",
            "content": user_input
        },
        {
            "role": "system",
            "content": json.dumps(table_schemas)
        }
    ]
    response_one = ai_response(messages)
    # 处理第一次ai结果
    # 如果模型返回的消息中tool call了
    if response_one.choices[0].message.tool_calls:
        # 获取第一个tool调用
        tool_call = response_one.choices[0].message.tool_calls[0]
        # 获取tool调用时的参数arg
        args = tool_call.function.arguments
        # log:sql_query
        print("tool call: " + tool_call.function.arguments)
        # 初始化函数结果字典
        function_result = {}
        # 根据tools调用的函数选择，执行对应的函数
        if tool_call.function.name == "execute_sql_query":
            function_result = execute_sql_query(**json.loads(args))
        # 构造tools消息并添加到消息列表中
        messages.append({
            "role": "tool",
            "content": f"{json.dumps(function_result)}",
            "tool_call_id": tool_call.id
        })
        # print(messages)
        response_two = ai_response(messages)
        result = response_two.choices[0].message.content
        print("response two: " + (result if result is not None else "No content"))
    return result


In [68]:
from zhipuai import ZhipuAI
import json

def ai_response(messages,tools):
    client = ZhipuAI(api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc") # 请填写您自己的APIKey
    response = client.chat.completions.create(
        model="glm-4", # 填写需要调用的模型名称
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    return response


In [ ]:
table_schemas = {
    "tender_index": "id int, title varchar(255), publish_time date, province varchar(100), source_platform varchar(100), info_type varchar(100), business_type varchar(100), industry varchar(100), detail_link varchar(255), detail_info_fetched tinyint",
    "tender_detail": "id int, tender_id int, tender_document_start_time varchar(255), tender_document_end_time varchar(255), question_deadline varchar(255), answer_announcement_time varchar(255), bid_submission_deadline varchar(255), bid_opening_time varchar(255), tenderer varchar(255), tender_contact varchar(255), contact_phone varchar(255), tender_agency varchar(255), tender_agency_contact varchar(255), tender_agency_contact_phone varchar(255), supervision_qualification_requirement text(65535), business_license_requirement varchar(255), chief_supervisor_qualification_requirement text(65535), consortium_bidding_requirement text(65535), project_name varchar(255), investment_project_code varchar(255), tender_project_name varchar(255), implementation_site varchar(255), funding_source varchar(255), tender_scope_and_scale text(65535), duration varchar(255), maximum_bid_price varchar(255), qualification_review_method varchar(255)",
    "tender_key_detail":"tender_id int, bid_price float, construction_duration int, construction_area float, construction_cost float, qualification_type varchar(50), qualification_level varchar(50), qualification_profession varchar(100), title_level varchar(50), education varchar(50), performance_requirements varchar(100), simultaneous_projects_limit varchar(100), qualification_profession_addition varchar(100)",
    "users": "id int, openid varchar(255), session_key varchar(255), nickname varchar(255), permission int, company_id int",
}

In [ ]:
system_message = {
    "role": "system",
    "content": "作为一个AI助手，当用户询问关于招标公告或监理招标的信息时，你会根据问题的具体内容选择恰当的数据表进行SQL查询。用户可能会使用不完整的名称或近似的文字，例如，'标'可能代表'招标公告'。对于关于招标公告的概览问题，优先在'tender_index'表中查询；若查询单个招标公告的重点信息或主要信息，则在tender_key_detail中查找；若查询单个招标公告的详细信息，则在'tender_detail'表中查找，但是尽量不要用精确文字查找，因为可能用户习惯使用部分名称而不是全称。在数据库中，日期的格式是 'YYYY-MM-DD'，例如 '2024-04-24'。查询结果将以对话形式返回，确保用户能够直观地理解查询内容和结果。"
}

In [45]:
#sql agent

system_message = {
    "role": "system",
    "content": """
    作为一个AI助手，协助用户，回答关于招标公告信息的问题。
    根据用户的问题，仅根据提供的相关数据库的信息，回答问题。
    如果相关信息不足，调用查询sql工具，查询数据库，依据返回的结果再回答问题。
    """
}

In [42]:
experience = """

涉及数据表信息的问题，建议流程：
1. 当你没有足够的表信息的时候，先获取当前数据库的所有表。
2. 选择可能和提问相关的表，获取表结构信息
3. 再获取表中前三行的数据。（注意：不要获取所有列，只查询相关的列）
4. 根据相关的表结构和数据，结合用户提问，生成一个SQL查询语句，查询数据库，返回结果。
5. 结合用户提问和返回的结果，生成一个回答。

你可用的工具有：
1. execute_sql_query(query) ：执行 MySQL SQL Query 查询并返回结果

切忌没有表结构context的情况下随意捏造生成SQL语句，容易导致SQL注入攻击。

"""


In [13]:
tools = [
        {
        "type": "function",
        "function": {
            "name": "execute_sql_query",
            "description": "执行 MySQL SQL 查询并返回结果。",
            "parameters": {
                "type": "object",
                "properties": {
                    "sql_query": {
                        "description": "MySQL的命令语句 SQL query。此语句必须符合 SQL 语法规范，确保可以正确执行以获取预期的数据库信息。",
                        "type": "string"
                    }
                },
                "required": ["sql_query"]
            },
            "returns": {
                "description": "The result of the SQL query execution, formatted as a string.",
                "type": "string"
            }
        }
    }
]


In [46]:
user_input = ' 最近三天，有什么标？ '

messages = [
    system_message,
    {"role": "system","content": experience},
    {"role": "user","content": "用户问题：" + user_input},
]

In [49]:
print(messages)

[{'role': 'system', 'content': '\n    作为一个AI助手，协助用户，回答关于招标公告信息的问题。\n    根据用户的问题，仅根据提供的相关数据库的信息，回答问题。\n    如果相关信息不足，调用查询sql工具，查询数据库，依据返回的结果再回答问题。\n    '}, {'role': 'system', 'content': '\n\n涉及数据表信息的问题，建议流程：\n1. 当你没有足够的表信息的时候，先获取当前数据库的所有表。\n2. 选择可能和提问相关的表，获取表结构信息\n3. 再获取表中前三行的数据。（注意：不要获取所有列，只查询相关的列）\n4. 根据相关的表结构和数据，结合用户提问，生成一个SQL查询语句，查询数据库，返回结果。\n5. 结合用户提问和返回的结果，生成一个回答。\n\n你可用的工具有：\n1. execute_sql_query(query) ：执行 MySQL SQL Query 查询并返回结果\n\n切忌没有表结构context的情况下随意捏造生成SQL语句，容易导致SQL注入攻击。\n\n'}, {'role': 'user', 'content': '用户问题： 最近三天，有什么标？ '}]


In [70]:
response = ai_response(messages,tools)
response

Completion(model='glm-4', created=1714040980, choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8602221806352866110', function=Function(arguments='{"sql_query":"SELECT * FROM bidding_announcement WHERE release_time \\u003e= DATE_SUB(CURDATE(), INTERVAL 3 DAY);"}', name='execute_sql_query'), type='function')]))], request_id='8602221806352866110', id='8602221806352866110', usage=CompletionUsage(prompt_tokens=344, completion_tokens=36, total_tokens=380))

In [52]:
response.choices[0].message.content

'这个问题需要查询招标公告信息数据库，找到最近三天内的招标公告。我需要使用工具来生成并执行相应的SQL查询语句。\n\n首先，我需要知道与招标公告相关的数据表信息。我将使用以下步骤来获取相关信息：\n\n1. 获取当前数据库的所有表。\n2. 确定与招标公告相关的表。\n3. 获取相关表的结构信息，以确定哪一列表示招标公告的发布时间。\n4. 根据表结构，生成一个查询最近三天招标公告的SQL查询语句。\n5. 执行该SQL查询，并获取结果。\n\n以下是我将执行的SQL查询的模板：\n\n```sql\nSELECT *\nFROM 招标公告表名\nWHERE 公告发布时间列名 >= NOW() - INTERVAL 3 DAY;\n```\n\n我将使用 `execute_sql_query` 函数来执行这个查询。\n\n由于我现在没有具体的表名和列名，我将使用以下假设的表名和列名来执行查询：\n\n- 假设招标公告表名为 `tender_announcements`。\n- 假设公告发布时间列名为 `announcement_date`。\n\n现在我将执行以下代码：\n\n```python\ntool_call(execute_sql_query("SELECT * FROM tender_announcements WHERE announcement_date >= NOW() - INTERVAL 3 DAY;"))\n```\n\n如果上述执行成功，我将得到最近三天的招标公告信息，并将结果返回给用户。\n\n请注意，实际中我需要先获取数据库的真实表名和列名，以上仅为示例。如果数据库信息与假设不同，我会根据实际情况调整查询语句。'

In [40]:
print(response.choices[0].message.tool_calls[0].function.arguments)

{"sql_query":"SELECT * FROM bidding WHERE bidding_date \u003e= DATE_SUB(CURDATE(), INTERVAL 3 DAY);"}


In [95]:
print(response.choices[0].message.role)

role = response.choices[0].message.role

args = response.choices[0].message.tool_calls[0].function.arguments

print(args)

query = json.loads(response.choices[0].message.tool_calls[0].function.arguments)

query


assistant
{"sql_query":"SELECT * FROM bidding_announcement WHERE release_time \u003e= DATE_SUB(CURDATE(), INTERVAL 3 DAY);"}


{'sql_query': 'SELECT * FROM bidding_announcement WHERE release_time >= DATE_SUB(CURDATE(), INTERVAL 3 DAY);'}

In [83]:
import mysql.connector

def execute_sql_query(sql_query:str):
    connection = mysql.connector.connect(
        host="gz-cdb-5scrcjb5.sql.tencentcdb.com",  # 数据库主机地址
        user="db",  # 数据库用户名
        password="dbdb905905",  # 数据库密码
        database="sele",  # 数据库名
        port=63432  # Corrected syntax for specifying the port argument
    )
    cursor = connection.cursor()
    # 执行SQL查询
    
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()  # 获取所有查询结果
        # 关闭连接
        cursor.close()
        connection.close()
        # 格式化结果为字符串，以便返回
        formatted_result = ", ".join([str(row) for row in result])
        return formatted_result
    except Exception as e:
        # 关闭连接
        cursor.close()
        connection.close()
        return f"Error executing SQL query: {e}"


In [58]:
connection = mysql.connector.connect(
    host="gz-cdb-5scrcjb5.sql.tencentcdb.com",  # 数据库主机地址
    user="db",  # 数据库用户名
    password="dbdb905905",  # 数据库密码
    database="sele",  # 数据库名
    port=63432  # Corrected syntax for specifying the port argument
)

In [ ]:
cursor = connection.cursor(**query)

In [61]:
query = "select count(*) from tender_index"

In [84]:
sql_result = execute_sql_query(**query)

In [ ]:
from zhipuai import ZhipuAI
import json
client = ZhipuAI(api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc") # 请填写您自己的APIKey
import mysql.connector

def execute_sql_query(sql_query:str):
    connection = mysql.connector.connect(
        host="gz-cdb-5scrcjb5.sql.tencentcdb.com",  # 数据库主机地址
        user="db",  # 数据库用户名
        password="dbdb905905",  # 数据库密码
        database="sele",  # 数据库名
        port=63432  # Corrected syntax for specifying the port argument
    )
    cursor = connection.cursor()
    # 执行SQL查询
    cursor.execute(sql_query)
    result = cursor.fetchall()  # 获取所有查询结果
    # 关闭连接
    cursor.close()
    connection.close()
    # 格式化结果为字符串，以便返回
    formatted_result = ", ".join([str(row) for row in result])
    return formatted_result

tools = [
        {
        "type": "function",
        "function": {
            "name": "execute_sql_query",
            "description": "执行 MySQL SQL 查询并返回结果。此功能利用预定义的表结构（table schemas），结合用户的数据输入，更好地理解根据用户输入需要查询哪些表，并根据这些表结构生成恰当的 SQL 查询语句，以满足用户的需求。这样可以确保生成的查询语句既符合数据库结构也符合用户查询的意图",
            "parameters": {
                "type": "object",
                "properties": {
                    "sql_query": {
                        "description": "将要在 MySQL 数据库上执行的命令语句 SQL query。此语句必须符合 SQL 语法规范，确保可以正确执行以获取预期的数据库信息。",
                        "type": "string"
                    }
                },
                "required": ["sql_query"]
            },
            "returns": {
                "description": "The result of the SQL query execution, formatted as a string.",
                "type": "string"
            }
        }
    }
]

system_message = {
    "role": "system",
    "content": "作为一个AI助手，当用户询问关于招标公告或监理招标的信息时，你会根据问题的具体内容选择恰当的数据表进行SQL查询。用户可能会使用不完整的名称或近似的文字，例如，'标'可能代表'招标公告'。对于关于招标公告的概览问题，优先在'tender_index'表中查询；若查询单个招标公告的重点信息或主要信息，则在tender_key_detail中查找；若查询单个招标公告的详细信息，则在'tender_detail'表中查找，但是尽量不要用精确文字查找，因为可能用户习惯使用部分名称而不是全称。在数据库中，日期的格式是 'YYYY-MM-DD'，例如 '2024-04-24'。查询结果将以对话形式返回，确保用户能够直观地理解查询内容和结果。"
}

def get_ai_response(user_input):
    table_schemas = {
        "tender_index": "id int, title varchar(255), publish_time date, province varchar(100), source_platform varchar(100), info_type varchar(100), business_type varchar(100), industry varchar(100), detail_link varchar(255), detail_info_fetched tinyint",
        "tender_detail": "id int, tender_id int, tender_document_start_time varchar(255), tender_document_end_time varchar(255), question_deadline varchar(255), answer_announcement_time varchar(255), bid_submission_deadline varchar(255), bid_opening_time varchar(255), tenderer varchar(255), tender_contact varchar(255), contact_phone varchar(255), tender_agency varchar(255), tender_agency_contact varchar(255), tender_agency_contact_phone varchar(255), supervision_qualification_requirement text(65535), business_license_requirement varchar(255), chief_supervisor_qualification_requirement text(65535), consortium_bidding_requirement text(65535), project_name varchar(255), investment_project_code varchar(255), tender_project_name varchar(255), implementation_site varchar(255), funding_source varchar(255), tender_scope_and_scale text(65535), duration varchar(255), maximum_bid_price varchar(255), qualification_review_method varchar(255)",
        "tender_key_detail":"tender_id int, bid_price float, construction_duration int, construction_area float, construction_cost float, qualification_type varchar(50), qualification_level varchar(50), qualification_profession varchar(100), title_level varchar(50), education varchar(50), performance_requirements varchar(100), simultaneous_projects_limit varchar(100), qualification_profession_addition varchar(100)",
        "users": "id int, openid varchar(255), session_key varchar(255), nickname varchar(255), permission int, company_id int",
    }
    messages = [
        system_message,
        {
            "role": "user",
            "content": user_input
        },
        {
            "role": "system",
            "content": json.dumps(table_schemas)
        }
    ]
    response_one = ai_response(messages)
    # 处理第一次ai结果
    # 如果模型返回的消息中tool call了
    if response_one.choices[0].message.tool_calls:
        # 获取第一个tool调用
        tool_call = response_one.choices[0].message.tool_calls[0]
        # 获取tool调用时的参数arg
        args = tool_call.function.arguments
        # log:sql_query
        print("tool call: " + tool_call.function.arguments)
        # 初始化函数结果字典
        function_result = {}
        # 根据tools调用的函数选择，执行对应的函数
        if tool_call.function.name == "execute_sql_query":
            function_result = execute_sql_query(**json.loads(args))
        # 构造tools消息并添加到消息列表中
        messages.append({
            "role": "tool",
            "content": f"{json.dumps(function_result)}",
            "tool_call_id": tool_call.id
        })
        # print(messages)
        response_two = ai_response(messages)
        result = response_two.choices[0].message.content
        print("response two: " + (result if result is not None else "No content"))
    return result


In [85]:
sql_result

"Error executing SQL query: 1146 (42S02): Table 'sele.bidding_announcement' doesn't exist"

In [96]:
messages.append({
    "role": role,
    "content": args
})

messages.append({
    "role": role,
    "content": sql_result
})

In [97]:
print(messages)

[{'role': 'system', 'content': '\n    作为一个AI助手，协助用户，回答关于招标公告信息的问题。\n    根据用户的问题，仅根据提供的相关数据库的信息，回答问题。\n    如果相关信息不足，调用查询sql工具，查询数据库，依据返回的结果再回答问题。\n    '}, {'role': 'system', 'content': '\n\n涉及数据表信息的问题，建议流程：\n1. 当你没有足够的表信息的时候，先获取当前数据库的所有表。\n2. 选择可能和提问相关的表，获取表结构信息\n3. 再获取表中前三行的数据。（注意：不要获取所有列，只查询相关的列）\n4. 根据相关的表结构和数据，结合用户提问，生成一个SQL查询语句，查询数据库，返回结果。\n5. 结合用户提问和返回的结果，生成一个回答。\n\n你可用的工具有：\n1. execute_sql_query(query) ：执行 MySQL SQL Query 查询并返回结果\n\n切忌没有表结构context的情况下随意捏造生成SQL语句，容易导致SQL注入攻击。\n\n'}, {'role': 'user', 'content': '用户问题： 最近三天，有什么标？ '}, {'role': 'assistant', 'content': '{"sql_query":"SELECT * FROM bidding_announcement WHERE release_time \\u003e= DATE_SUB(CURDATE(), INTERVAL 3 DAY);"}'}, {'role': 'assistant', 'content': "Error executing SQL query: 1146 (42S02): Table 'sele.bidding_announcement' doesn't exist"}]


In [103]:
response = ai_response(messages,tools)

response

Completion(model='glm-4', created=1714042350, choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8602217305227053920', function=Function(arguments='{"sql_query":"SHOW TABLES;"}', name='execute_sql_query'), type='function')]))], request_id='8602217305227053920', id='8602217305227053920', usage=CompletionUsage(prompt_tokens=402, completion_tokens=19, total_tokens=421))

In [104]:
args = response.choices[0].message.tool_calls[0].function.arguments

print(args)

query = json.loads(args)
print(query)

{"sql_query":"SHOW TABLES;"}
{'sql_query': 'SHOW TABLES;'}


In [105]:
sql_result = execute_sql_query(**query)
sql_result

"('announcement_catalog',), ('announcement_labels',), ('company',), ('company_qualification_level',), ('company_qualification_type',), ('education_level',), ('professional_title_level',), ('proofread_detail',), ('raw_tender_key_detail',), ('registration_certificate_profession',), ('task',), ('task_status',), ('tender_detail',), ('tender_detail_html',), ('tender_detail_html_temp',), ('tender_detail_temp',), ('tender_detail_tier_one',), ('tender_index',), ('tender_key_detail',), ('tender_scores_temp',), ('users',)"

In [106]:
messages.append({
    "role": role,
    "content": args
})

messages.append({
    "role": role,
    "content": sql_result
})

In [109]:
print(messages)

[{'role': 'system', 'content': '\n    作为一个AI助手，协助用户，回答关于招标公告信息的问题。\n    根据用户的问题，仅根据提供的相关数据库的信息，回答问题。\n    如果相关信息不足，调用查询sql工具，查询数据库，依据返回的结果再回答问题。\n    '}, {'role': 'system', 'content': '\n\n涉及数据表信息的问题，建议流程：\n1. 当你没有足够的表信息的时候，先获取当前数据库的所有表。\n2. 选择可能和提问相关的表，获取表结构信息\n3. 再获取表中前三行的数据。（注意：不要获取所有列，只查询相关的列）\n4. 根据相关的表结构和数据，结合用户提问，生成一个SQL查询语句，查询数据库，返回结果。\n5. 结合用户提问和返回的结果，生成一个回答。\n\n你可用的工具有：\n1. execute_sql_query(query) ：执行 MySQL SQL Query 查询并返回结果\n\n切忌没有表结构context的情况下随意捏造生成SQL语句，容易导致SQL注入攻击。\n\n'}, {'role': 'user', 'content': '用户问题： 最近三天，有什么标？ '}, {'role': 'assistant', 'content': '{"sql_query":"SELECT * FROM bidding_announcement WHERE release_time \\u003e= DATE_SUB(CURDATE(), INTERVAL 3 DAY);"}'}, {'role': 'assistant', 'content': "Error executing SQL query: 1146 (42S02): Table 'sele.bidding_announcement' doesn't exist"}, {'role': 'assistant', 'content': '{"sql_query":"SHOW TABLES;"}'}, {'role': 'assistant', 'content': "('announcement_catalog',), ('announcement_labels',), ('company',), ('company_q

In [108]:
response = ai_response(messages,tools)

response

Completion(model='glm-4', created=1714042447, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='很抱歉，目前无法直接回答您的问题，因为系统中没有找到与招标公告相关的表。我们建议您直接访问相关招标网站或者联系招标方以获取最新的招标信息。如果有其他问题，我们会尽力帮助您。', role='assistant', tool_calls=None))], request_id='8602217167788093963', id='8602217167788093963', usage=CompletionUsage(prompt_tokens=543, completion_tokens=49, total_tokens=592))

In [ ]:
[{'role': 'system', 'content': '\n    作为一个AI助手，协助用户，回答关于招标公告信息的问题。\n    根据用户的问题，仅根据提供的相关数据库的信息，回答问题。\n    如果相关信息不足，调用查询sql工具，查询数据库，依据返回的结果再回答问题。\n    '}, {'role': 'system', 'content': '\n\n涉及数据表信息的问题，建议流程：\n1. 当你没有足够的表信息的时候，先获取当前数据库的所有表。\n2. 选择可能和提问相关的表，获取表结构信息\n3. 再获取表中前三行的数据。（注意：不要获取所有列，只查询相关的列）\n4. 根据相关的表结构和数据，结合用户提问，生成一个SQL查询语句，查询数据库，返回结果。\n5. 结合用户提问和返回的结果，生成一个回答。\n\n你可用的工具有：\n1. execute_sql_query(query) ：执行 MySQL SQL Query 查询并返回结果\n\n切忌没有表结构context的情况下随意捏造生成SQL语句，容易导致SQL注入攻击。\n\n'}, {'role': 'user', 'content': '用户问题： 最近三天，有什么标？ '}, {'role': 'assistant', 'content': '{"sql_query":"SELECT * FROM bidding_announcement WHERE release_time \\u003e= DATE_SUB(CURDATE(), INTERVAL 3 DAY);"}'}, {'role': 'assistant', 'content': "Error executing SQL query: 1146 (42S02): Table 'sele.bidding_announcement' doesn't exist"}, {'role': 'assistant', 'content': '{"sql_query":"SHOW TABLES;"}'}, {'role': 'assistant', 'content': "('announcement_catalog',), ('announcement_labels',), ('company',), ('company_qualification_level',), ('company_qualification_type',), ('education_level',), ('professional_title_level',), ('proofread_detail',), ('raw_tender_key_detail',), ('registration_certificate_profession',), ('task',), ('task_status',), ('tender_detail',), ('tender_detail_html',), ('tender_detail_html_temp',), ('tender_detail_temp',), ('tender_detail_tier_one',), ('tender_index',), ('tender_key_detail',), ('tender_scores_temp',), ('users',)"}]